## this file will be using tools to search for flights and book tickets using LLM

### what it does
- Chat with an LLM
- tool 1: search flight - queries through the mock database for availability
- tool 2: reserves a seat by writing in-memory(dict) and returns a confirmation code

In [1]:
## import
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import json
import random
import string

In [2]:
## load the data
data  = pd.read_csv("flights.csv")
data.head(2)

,flight_no,origin,destination,date,depart,arrive,price,cabin,seats_left
0,QF401,SYD,MEL,2025-09-01,07:00,08:30,180,economy,5
1,QF402,SYD,MEL,2025-09-01,09:00,10:30,220,business,3


In [3]:
## initialization
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

openai_client = OpenAI()

MODEL = 'gpt-4o-mini'

In [4]:
## define the system prompt
system_prompt = """
You are a very helpful assistant for an airline called SydAirAI.
Always answer in 1 sentence. If unsure, say so plainly.

Before calling any tool, NORMALIZE user inputs:
- Map common city names to IATA codes (Sydney→SYD, Melbourne→MEL, Brisbane→BNE, Perth→PER, Adelaide→ADL, Canberra→CBR).
- Uppercase IATA codes.
- Convert any date to YYYY-MM-DD.
- If the user says 'any' cabin or doesn't specify cabin, use 'any' for cabin.
- If seats not specified, assume 1.

If a city is ambiguous (e.g., “Newcastle”), ask a clarifying question.
"""

In [5]:
## generic funtions to call the tools

## tool 1: availability of the files
def check_availability(
    origin: str,
    destination: str,
    date: str,
    cabin: str,
    required_seats: int = 1,
    max_results: int=3
):
    try:
        ## normalize the input values
        origin = origin.strip().upper()
        destination = destination.strip().upper()
        date = date.strip()
        cabin = cabin.strip().lower()
        required_seats = int(required_seats)

        ## query the data
        df = data.query("origin == @origin and destination == @destination and date == @date and seats_left >= @required_seats and cabin == @cabin")

        ## sort by the cheapest and earliest departure
        df = df.sort_values(["price", "depart"]).head(max_results)
    
        ## if search result is empty
        if df.empty:
            return {
                "ok": False,
                "message": "No flight found at this moment"
            }
        return {
            "ok": True,
            "flights": df.to_dict(orient="records")
        }
    except Exception as e:
        return{
            "ok":False,
            "error": str(e)
        }



## tool 2: Book a flight

## in-memory storage for booking
bookings = {}

def book_flight(
    flight_no: str, 
    name: str, 
    email: str, 
    date: str,
    seats_asked: int,
    cabin: str
):
    try:
        flight_no = flight_no.strip().upper()
        name = name.strip()
        email = email.strip()
        date = date.strip()
        seats_asked = int(seats_asked)
        cabin = cabin.strip()

        ## guardrail if asked_seat is minus or zero
        if seats_asked <= 0:
            return{
                "ok":False,
                "message": "seats should be for at least 1 person."
            }
            
        ## find the flight
        df = data.query("flight_no == @flight_no and date == @date")
        if df.empty:
            return {
                "ok": False,
                "message": "No flights found with the search criteria"
            }

        row = df.iloc[0]
            
        ## availability check
        if cabin and row["cabin"] != cabin.strip().lower():
            return{
                "ok":False,
                "message": f"Cabin mismatch. Flight cabin is '{row['cabin']}'."
            }
        
        seats_left = int(row["seats_left"])
        if seats_left<seats_asked:
            return {
                "ok":False,
                "message": "Not enough seats available"
            }
            
        ## reduce the seats_left once booked - main data
        idx = data.index[(data.flight_no == flight_no) & (data.date==date)][0]
        data.at[idx, "seats_left"] = int(data.at[idx, "seats_left"]) - int(seats_asked)

        ## generate a sample PNR
        pnr = "".join(random.choices(string.ascii_uppercase +  string.digits, k=6))

        ## sabe the booking
        booking = {
            "pnr": pnr,
            "flight_no":flight_no,
            "date": date,
            "origin": row.origin,
            "destination": row.destination,
            "depart": row.depart,
            "arrive": row.arrive,
            "cabin": row.cabin,
            "name": name,
            "email": email,
            "seats": seats_asked,
            "price_total": float(row.price) * seats_asked
        }
        bookings[pnr] = booking

        return {
            "ok":True,
            "booking": booking
        }
        
    except Exception as e:
        return {
            "ok":False,
            "error": str(e)
        }

In [6]:
## following the particular dictionary structure to describe the functions as tools

## tool registration - 1: Availability tool
availability_tool = {
    "name": "check_availability",
    "description": """Check flight availability. Always convert city names to IATA codes
                      (e.g., Sydney→SYD, Melbourne→MEL) and pass those to the tool.
                      Convert dates to YYYY-MM-DD. If the customer says 'any' cabin or
                      doesn't specify, pass cabin='any'. Assume 1 seat if not given.
                      The customer might ask 'I want to travel from sydney to melbourne on 22 of july in economy call. I am just travelling by myself'""",
    "parameters":{
        "type": "object",
        "properties":{
            "origin":{
                "type": "string",
                "description": "The city the customer is travelling from"
            },
            "destination":{
                "type": "string",
                "description": "The city the customer wants to go to"
            },
            "date":{
                "type":"string",
                "description": "The date the customer is willing to travel from the origin city"
            },
            "cabin":{
                "type": "string",
                "description": "The type of cabin the customer is willing to travel in. for example: business and economy"
            },
            "required_seats":{
                "type": "integer",
                "description": "The number of seats the customer wants to book"
            },
            "max_results":{
                "type":"integer",
                "description": "the maximum number of response that can be obtained from a search with the check_availability function"
            },
        },
        "required":["origin", "destination", "date", "cabin", "required_seats"],
        "additionalProperties": False
    },
}

## tool registration - 2: Booking tool
booking_tool = {
    "name": "book_flight",
    "description": """ Book the flight of the customer when they choose a specific flight.
                       call this function when the customer confirms that they want to make a reservation.
                       this requires a flight number, travel date, passenger name and email.
                       reserve the seats with cabin type.
                       Example: I want to book 2 economy tickets on AB123 for july 22 under Mostafa Protik""",
    "parameters":{
        "type": "object",
        "properties":{
            "flight_no":{
                "type": "string",
                "description": "the selected flight for the customer to travel"
            },
            "name":{
                "type": "string",
                "description": "The name of the customer under that he/she will be travelling"
            },
            "email":{
                "type":"string",
                "description": "The email address of the customer that is used in booking"
            },
            "date":{
                "type": "string",
                "description": "The date of booking when the customer wants to travel"
            },
            "seats_asked":{
                "type": "integer",
                "description": "The total number of seats the customer wants to reserve"
            },
            "cabin":{
                "type":"string",
                "description": "the type of cabin the customer wishes to reserve"
            },
        },
        "required":["flight_no", "name", "email", "date", "seats_asked", "cabin"],
        "additionalProperties": False
    },
}

tools = [
    {"type": "function", "function": availability_tool},
    {"type": "function", "function": booking_tool}
]

In [7]:
def _run_tool(name, args):
    if name=="check_availability":
        return check_availability(**args)
    if name=="book_flight":
        return book_flight(**args)
    return {
        "ok": False,
        "message": f"unknown tool: {name}"
    }

In [8]:
def chat(message, history):
    messages = [
        {"role":"system", "content": system_prompt}
    ]

    for msg in history:
        messages.append(msg)
    messages.append({"role": "user", "content": message})

    result = openai_client.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools,
        tool_choice = "auto"
    )
    ## if tool is required - not the final answer, rather an instruction to call the tools
    if result.choices[0].finish_reason=="tool_calls":
        ## capture assistants request
        assistant_message = result.choices[0].message
        messages.append({"role":"assistant", "tool_calls": assistant_message.tool_calls})
        ## run tool manually and append outputs
        tools_msgs = handle_tool_call(assistant_message)
        messages.extend(tools_msgs)

        ## finalize
        followup = openai_client.chat.completions.create(
            model = MODEL,
            messages = messages
        )
        return followup.choices[0].message.content

    # no tool call path
    return result.choices[0].message.content
        

In [9]:
## tool layer
def handle_tool_call(message):
    tool_messages = []
    ## looping over all the requested tools
    for tool_call in message.tool_calls:
        name = tool_call.function.name # get the name of the tool
        args = json.loads(tool_call.function.arguments) # get the args to run the function
        result = _run_tool(name, args) # run the function in python
        tool_messages.append({
            "role":"tool",
            "tool_call_id": tool_call.id,
            "name": name,
            "content": json.dumps(result)
        })
    return tool_messages

In [10]:
gr.ChatInterface(
    fn=chat,
    type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
